# Homework 8

## Data preparation

- In this homework, we'll build a model for predicting if we have an image of a dog or a cat. For this, we will use the "Dogs & Cats" dataset that can be downloaded from Kaggle.

- You need to download the train.zip file.

- If you have troubles downloading from Kaggle, use this link instead:

- `wget https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats/train.zip`

In [ ]:
import os 
import shutil

In [ ]:
os.getcwd()

In [ ]:
# !wget https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats/train.zip

In [ ]:
os.listdir()

In [ ]:
# !unzip train.zip -d .

In [ ]:
os.listdir('./train')

In [ ]:
# os.mkdir('./train/cats')
# os.mkdir('./train/dogs')

In [ ]:
# os.mkdir('./val')
# os.mkdir('./val/cats')
# os.mkdir('./val/dogs')

# cat_train_files = [f'cat.{i}.jpg' for i in range(10000)]
# cat_val_files = [f'cat.{i}.jpg' for i in range(10000, 12500)]
# dog_train_files = [f'dog.{i}.jpg' for i in range(10000)]
# dog_val_files = [f'dog.{i}.jpg' for i in range(10000, 12500)]

# os.chdir(os.getcwd()+'/train')
# os.getcwd()

# for file in cat_train_files:
#     shutil.move(f'./{file}', './cats/')

# for file in dog_train_files:
#     shutil.move(f'./{file}', './dogs')

In [ ]:
os.getcwd()

In [ ]:
# for file in cat_val_files:
#     shutil.move(f'./{file}', '../val/cats')

# for file in dog_val_files:
#     shutil.move(f'./{file}', '../val/dogs')

In [ ]:
os.getcwd()

In [ ]:
# os.chdir('..')

# !mkdir hw-data

# shutil.move('./train', './hw-data/')
# shutil.move('./val', './hw-data/')

## Model

develop the model with following structure:

The shape for input should be `(150, 150, 3)`
Next, create a covolutional layer (Conv2D):
```
Use 32 filters
Kernel size should be (3, 3) (that's the size of the filter)
Use 'relu' as activation
Reduce the size of the feature map with max pooling (MaxPooling2D)
Set the pooling size to (2, 2)

Turn the multi-dimensional result into vectors using a Flatten layer
Next, add a Dense layer with 64 neurons and 'relu' activation
Finally, create the Dense layer with 1 neuron - this will be the output
The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use SGD with the following parameters:

SGD(lr=0.002, momentum=0.8)
```

In [1]:
import tensorflow as tf
from tensorflow import keras as keras
from tensorflow.keras import layers

2021-11-26 18:32:17.273881: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
model = keras.Sequential()
model.add(keras.Input(shape= (150, 150, 3)))
model.add(layers.Conv2D(filters= 32, kernel_size= (3, 3), activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size= (2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation= 'relu'))
model.add(layers.Dense(1, activation= 'sigmoid'))

2021-11-26 18:32:46.972427: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-26 18:32:46.975142: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-26 18:32:47.090369: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-26 18:32:47.091666: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1050 Ti computeCapability: 6.1
coreClock: 1.62GHz coreCount: 6 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 104.43GiB/s
2021-11-26 18:32:47.091751: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-11-26 18:32:47.544861: I tensorflow/stream_executor/platform/d

### Question 1

Since we have a binary classification problem, what is the best loss function for us?

In [3]:
sgd = keras.optimizers.SGD(learning_rate=0.02, momentum= 0.8)
model.compile(optimizer= sgd, loss= keras.losses.BinaryCrossentropy(), metrics=["accuracy"])

### Question 2

What's the total number of parameters of the model? You can use the summary method for that.

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 175232)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                11214912  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
_________________________________________________________________


## Generators and Training

For the next two questions, use the following data generator for both train and validation:

    `ImageDataGenerator(rescale=1./255)`  

We don't need to do any additional pre-processing for the images.
    When reading the data from train/val directories, check the class_mode parameter. Which value should it be for a binary classification problem?
    Use `batch_size=20`

For training use `.fit()` with the following params:

```
model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)
```

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
image_size = (150, 150)
batch_size = 20

In [7]:
train_gen = ImageDataGenerator(rescale=1./255)

train_ds = train_gen.flow_from_directory('./hw-data/train/', target_size= image_size, batch_size= batch_size, class_mode= 'binary', shuffle= True)

Found 20000 images belonging to 2 classes.


In [8]:
val_gen = ImageDataGenerator(rescale=1./255)

val_ds = val_gen.flow_from_directory('./hw-data/val/', target_size= image_size, batch_size= batch_size, class_mode= 'binary', shuffle= True)

Found 5000 images belonging to 2 classes.


In [9]:
# cb = [
#     keras.callbacks.ModelCheckpoint(
#         "cat_dog.h5",
#         monitor="val_accuracy",
#         save_best_only=False,
#         mode='auto'
#     )
# ]


history = model.fit(
    train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=val_ds,
    validation_steps=50
)

2021-11-26 18:33:06.522152: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-26 18:33:06.550850: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2299965000 Hz


Epoch 1/10


2021-11-26 18:33:07.850504: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-11-26 18:33:12.405589: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-11-26 18:33:22.979225: W tensorflow/stream_executor/gpu/asm_compiler.cc:98] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You do not need to update to CUDA 9.2.88; cherry-picking the ptxas binary is sufficient.


100/100 [==============================] - 70s 531ms/step - loss: 0.7816 - accuracy: 0.5331 - val_loss: 0.6901 - val_accuracy: 0.5940
Epoch 2/10
100/100 [==============================] - 46s 461ms/step - loss: 0.6916 - accuracy: 0.5345 - val_loss: 0.6867 - val_accuracy: 0.5150
Epoch 3/10
100/100 [==============================] - 40s 400ms/step - loss: 0.6892 - accuracy: 0.5495 - val_loss: 0.6903 - val_accuracy: 0.5430
Epoch 4/10
100/100 [==============================] - 35s 352ms/step - loss: 0.6838 - accuracy: 0.5831 - val_loss: 0.6863 - val_accuracy: 0.5680
Epoch 5/10
100/100 [==============================] - 72s 722ms/step - loss: 0.6825 - accuracy: 0.5766 - val_loss: 0.6801 - val_accuracy: 0.5690
Epoch 6/10
100/100 [==============================] - 38s 376ms/step - loss: 0.6783 - accuracy: 0.5718 - val_loss: 0.6853 - val_accuracy: 0.5570
Epoch 7/10
100/100 [==============================] - 27s 272ms/step - loss: 0.6660 - accuracy: 0.6037 - val_loss: 0.6726 - val_accuracy: 0.5

### Question 3

What is the median of training accuracy for this model?

In [10]:
import numpy as np

In [11]:
train_median = np.median(history.history['accuracy'])
print(f'Median training accuracy is: {train_median:.3f}')

Median training accuracy is: 0.581


### Question 4

What is the standard deviation of training loss for this model?

In [12]:
std_train_loss = np.std(history.history['loss'])
print(f'Standard deviation of training loss is: {std_train_loss:.3f}')

Standard deviation of training loss is: 0.016


## Data Augmentation

For the next two questions, we'll generate more data using data augmentations.

Add the following augmentations to your training data generator:

- rotation_range=40,
- width_shift_range=0.2,
- height_shift_range=0.2,
- shear_range=0.2,
- zoom_range=0.2,
- horizontal_flip=True,
- fill_mode='nearest'


In [13]:
train_gen_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [14]:
train_ds_aug = train_gen_aug.flow_from_directory('./hw-data/train/', target_size= image_size, batch_size= batch_size, class_mode= 'binary', shuffle= True)

Found 20000 images belonging to 2 classes.


Train our model for 10 more epochs using the same code as previously. Make sure you don't re-create the model - we want to continue training the model we already started training.

In [15]:
history_aug = model.fit(
train_ds_aug,
steps_per_epoch=100,
initial_epoch= 10,
epochs=20,
validation_data=val_ds,
validation_steps=50,
shuffle= True
)

Epoch 11/20
100/100 [==============================] - 33s 335ms/step - loss: 0.6871 - accuracy: 0.5570 - val_loss: 0.6698 - val_accuracy: 0.5940
Epoch 12/20
100/100 [==============================] - 31s 310ms/step - loss: 0.6830 - accuracy: 0.5610 - val_loss: 0.6723 - val_accuracy: 0.5690
Epoch 13/20
100/100 [==============================] - 29s 291ms/step - loss: 0.6757 - accuracy: 0.5845 - val_loss: 0.6762 - val_accuracy: 0.5780
Epoch 14/20
100/100 [==============================] - 27s 271ms/step - loss: 0.6826 - accuracy: 0.5615 - val_loss: 0.6676 - val_accuracy: 0.5900
Epoch 15/20
100/100 [==============================] - 26s 262ms/step - loss: 0.6834 - accuracy: 0.5585 - val_loss: 0.6677 - val_accuracy: 0.5970
Epoch 16/20
100/100 [==============================] - 26s 262ms/step - loss: 0.6739 - accuracy: 0.5830 - val_loss: 0.6593 - val_accuracy: 0.6090
Epoch 17/20
100/100 [==============================] - 26s 257ms/step - loss: 0.6857 - accuracy: 0.5565 - val_loss: 0.6699 -

### Question 5

What is the mean of validation loss for the model trained with augmentations?

In [16]:
mean_val_loss = np.mean(history_aug.history['val_loss'])
print(f'Mean of validation loss is: {mean_val_loss:.3f}')

Mean of validation loss is: 0.669


### Question 6

What's the average of validation accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

In [17]:
mean_val_acc = np.mean(history_aug.history['val_accuracy'][-5])
print(f'Mean of validation accuracy is: {mean_val_acc:.3f}')

Mean of validation accuracy is: 0.609
